# Prior distributions in PEtab

This notebook gives a brief overview of the prior distributions in PEtab and how they are represented in the PEtab library.

Prior distributions are used to specify the prior knowledge about the parameters.
Parameter priors are specified in the parameter table. A prior is defined by its type and its parameters.
Each prior type has a specific set of parameters. For example, the normal distribution has two parameters: the mean and the standard deviation.

There are two types of priors in PEtab - objective priors and initialization priors:

* *Objective priors* are used to specify the prior knowledge about the parameters that are to be estimated. They will enter the objective function of the optimization problem. They are specified in the `objectivePriorType` and `objectivePriorParameters` columns of the parameter table.
* *Initialization priors* can be used as a hint for the optimization algorithm. They will not enter the objective function. They are specified in the `initializationPriorType` and `initializationPriorParameters` columns of the parameter table.




In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from petab.v1.C import *
from petab.v1.distributions import *

sns.set_style(None)


def plot(distr: Distribution, ax=None):
    """Visualize a distribution."""
    if ax is None:
        fig, ax = plt.subplots()

    sample = distr.sample(10000)

    # pdf
    xmin = min(sample.min(), distr.lb_scaled if distr.bounds is not None else sample.min())
    xmax = max(sample.max(), distr.ub_scaled if distr.bounds is not None else sample.max())
    x = np.linspace(xmin, xmax, 500)
    y = distr.pdf(x)
    ax.plot(x, y, color='red', label='pdf')

    sns.histplot(sample, stat='density', ax=ax, label="sample")

    # bounds
    if distr.bounds is not None:
        for bound in (distr.lb_scaled, distr.ub_scaled):
            if bound is not None and np.isfinite(bound):
                ax.axvline(bound, color='black', linestyle='--', label='bound')

    ax.set_title(str(distr))
    ax.set_xlabel('Parameter value on the parameter scale')
    ax.grid(False)
    handles, labels = ax.get_legend_handles_labels()
    unique_labels = dict(zip(labels, handles))
    ax.legend(unique_labels.values(), unique_labels.keys())
    plt.show()

The basic distributions are the uniform, normal, Laplace, log-normal, and log-laplace distributions:


In [ ]:
plot(Uniform(0, 1))
plot(Normal(0, 1))
plot(Laplace(0, 1))
plot(LogNormal(0, 1))
plot(LogLaplace(1, 0.5))

If a parameter scale is specified (`parameterScale=lin|log|log10` not a `parameterScale*`-type distribution), the sample is transformed accordingly (but not the distribution parameters):


In [ ]:
plot(Normal(10, 2, transformation=LIN))
plot(Normal(10, 2, transformation=LOG))
# Note that the log-normal distribution is different from a log-transformed normal distribution:
plot(LogNormal(10, 2, transformation=LIN))

On the log-transformed parameter scale, `Log*` and `parameterScale*` distributions are equivalent:

In [ ]:
plot(LogNormal(10, 2, transformation=LOG))
plot(ParameterScaleNormal(10, 2))

Prior distributions can also be defined on the parameter scale by using the types `parameterScaleUniform`, `parameterScaleNormal` or `parameterScaleLaplace`. In these cases, 1) the distribution parameter are interpreted on the transformed parameter scale, and 2) a sample from the given distribution is used directly, without applying any transformation according to `parameterScale` (this implies, that for `parameterScale=lin`, there is no difference between `parameterScaleUniform` and `uniform`):

In [ ]:
plot(Uniform(0, 1, transformation=LOG10))
plot(ParameterScaleUniform(0, 1, transformation=LOG10))

plot(Uniform(0, 1, transformation=LIN))
plot(ParameterScaleUniform(0, 1, transformation=LIN))


To prevent the sampled parameters from exceeding the bounds, the sampled parameters are clipped to the bounds. The bounds are defined in the parameter table. Note that the current implementation does not support sampling from a truncated distribution. Instead, the samples are clipped to the bounds. This may introduce unwanted bias, and thus, should only be used with caution (i.e., the bounds should be chosen wide enough):

In [ ]:
plot(Normal(0, 1, bounds=(-4, 4)))  # negligible clipping-bias at 4 sigma
plot(Uniform(0, 1, bounds=(0.1, 0.9)))  # significant clipping-bias

Further distribution examples:

In [ ]:
plot(Normal(10, 1, bounds=(6, 14), transformation="log10"))
plot(ParameterScaleNormal(10, 1, bounds=(10**6, 10**14), transformation="log10"))
